In [4]:
import json
import numpy as np
import pandas as pd

import os
from sklearn.externals import joblib


def init(model_path="model.pkl"):
    global model
    model = joblib.load(model_path)


def predict(data):
    try:
        #data = np.array(json.loads(data))
        result = model.predict(data)
        # You can return any data type, as long as it is JSON serializable.
        return result
    except Exception as e:
        error = str(e)
        return error


def predict_proba(data):
    try:
        #data = np.array(json.loads(data))
        result = model.predict_proba(data)
        # You can return any data type, as long as it is JSON serializable.
        return result
    except Exception as e:
        error = str(e)
        return error


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [5]:
id_name = "ID"
target = "TARGET"
model_path = "d_scs_model.pkl"
test_file = "test_d_scs.csv"


init(model_path)

test_df = pd.read_csv(test_file)
y_true = test_df[target]

y_pred = predict(test_df.drop([target],axis=1))

print("accuracy:")
print(np.mean(y_pred==y_true))

from sklearn import metrics

y_pred_prob = predict_proba(test_df.drop([target],axis=1))

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred_prob[:,1], pos_label=1)
print("auc:")
print(metrics.auc(fpr, tpr))

accuracy:
0.9702709813207051
auc:
0.931477251888035


In [8]:
## predcict test dataset on Kaggle

kaggle_submit_head = [id_name, target]

kaggle_test_file = "kaggle-test-d_scs.csv"
kaggle_pred_file = "kaggle-test-d_scs-predictions.csv"

kaggle_test_df = pd.read_csv(kaggle_test_file)
kaggle_test_id = np.array(kaggle_test_df[id_name])
#print(kaggle_test_df.head())
print(kaggle_test_df.shape)

count = 0
kaggle_pred = []
pred_neg_prpb = []
pred_pos_prpb = []

for i in range(10):
    cur_pred = predict(kaggle_test_df.iloc[i:i+1,])[0]
    cur_id = kaggle_test_id[i]
    kaggle_pred.append([cur_id, cur_pred])

print(kaggle_pred)


(75818, 370)
[[2, array([0])], [5, array([0])], [6, array([0])], [7, array([0])], [9, array([0])], [11, array([0])], [12, array([0])], [15, array([0])], [16, array([0])], [17, array([0])]]


In [18]:
assert kaggle_y_pred.shape == kaggle_test_id.shape

kaggle_pred_df = pd.DataFrame({kaggle_submit_head[0]: kaggle_test_id, kaggle_submit_head[1]: kaggle_y_pred})

print(kaggle_pred_df.head())

 
kaggle_pred_df.to_csv(kaggle_pred_file, index=False)

(75818,)
   ID  TARGET
0   2       0
1   5       0
2   6       0
3   7       0
4   9       0
